In [1]:
import pandas as pd 
import numpy as np 
import json
import pyomo.environ as pyo

In [17]:
# load json as list 
with open('data/demo.json') as f:
    data = json.load(f)
df_params = pd.DataFrame([param for param in data])
df_params['deal_sales_lower_bound'] = 2 * df_params['sales_at_lower_bound']
df_params['deal_sales_upper_bound'] = 2 * df_params['sales_at_upper_bound']

# to json
df_params.to_json('data/demo_with_deal.json', orient='records')


In [18]:
# 平时销量与价格的关系
df_params['slope'] = (df_params['sales_at_upper_bound'] - df_params['sales_at_lower_bound']) / (df_params['price_upper_bound'] - df_params['price_lower_bound'])
df_params['intercept'] = df_params['sales_at_lower_bound'] - df_params['slope'] * df_params['price_lower_bound']

# deal 价格下的销量
df_params['deal_slope'] = (df_params['deal_sales_upper_bound'] - df_params['deal_sales_lower_bound']) / (df_params['price_upper_bound'] - df_params['price_lower_bound'])
df_params['deal_intercept'] = df_params['deal_sales_lower_bound'] - df_params['deal_slope'] * df_params['price_lower_bound']

- deal 天数 
    - 1,2,3,4,7,8,9,10 
- 产品是否deal n 
- 产品价格 
    - 2**10 

In [4]:
from amplpy import modules

In [44]:
# df_params = df_params[:17].copy()

In [49]:
n_product = len(df_params)
model = pyo.ConcreteModel()

def price_bounds_rule(model, i):
    return df_params.loc[i, 'price_lower_bound'], df_params.loc[i, 'price_upper_bound']

model.prices = pyo.Var(range(n_product), domain=pyo.PositiveIntegers, bounds=price_bounds_rule)

# 每个产品是否做deal
model.deal = pyo.Var(range(n_product), domain=pyo.NonNegativeIntegers, bounds=(0, 10))

# set product 1 deal to 1 
model.deal[0].fix(10.0)
# def Peak_Rule(m, i):
#     return model.deal[0] >= model.deal[i]

# model.main_deal = pyo.Constraint([i for i in model.deal], rule=Peak_Rule)

# # 整个产品deal的天数 
# model.A = pyo.Set(initialize=['LD', 'BD'])
# lb = {'LD': 0, 'BD': 0}
# ub = {'LD': 4, 'BD': 1}

# def fb(model, i):
#     return (lb[i], ub[i])

# model.deal_days = pyo.Var(model.A, bounds=fb, domain=pyo.NonNegativeIntegers)

# 总利润
def total_profit_rule(model):
    a = df_params['slope']
    b = df_params['intercept']
    a_d = df_params['deal_slope']
    b_d = df_params['deal_intercept']
    total_profit = 0
    for i in range(n_product):
        # 日常每天利润
        fba_commission = model.prices[i] * 0.15
        manual_fee = model.prices[i] * 0.06
        storage_fee = model.prices[i] * 0.02

        # 每个产品的利润 日常
        product_income = model.prices[i] * (1 - df_params['damage_rate'][i]) - df_params['latest_amazon_delivery_fee_usd'][i] - \
                            fba_commission - df_params['vat_local'][i] - df_params['promotion_discount'][i] - \
                            df_params['ppc_cost'][i] - manual_fee - storage_fee
        product_profit_rmb =  product_income * df_params['exchange_rate'][i] - df_params['purchase_cost'][i] - \
                            df_params['shipping_and_tax'][i]

        # 每个产品的利润 deal
        product_income_deal = model.prices[i] * 0.85 * (1 - df_params['damage_rate'][i]) - df_params['latest_amazon_delivery_fee_usd'][i] - \
                            fba_commission - df_params['vat_local'][i] - df_params['promotion_discount'][i] - \
                            df_params['ppc_cost'][i] - manual_fee - storage_fee
        product_profit_rmb_deal = product_income_deal * df_params['exchange_rate'][i] - df_params['purchase_cost'][i] - \
                            df_params['shipping_and_tax'][i]
        
        # 日常销量
        product_sales = a[i] * model.prices[i] + b[i]
        
        # deal销量
        product_sales_deal = a_d[i] * model.prices[i] + b_d[i]

        # 总利润
        # all_deal_days = model.deal_days['LD'] + model.deal_days['BD']*7
        total_profit += (30 - model.deal[i]) * product_profit_rmb * product_sales \
            + model.deal[i] * product_profit_rmb_deal * product_sales_deal
            # - model.deal_days['LD']*150 - model.deal_days['BD']*300 # deal费用
            
    return total_profit

model.total_profit = pyo.Objective(rule=total_profit_rule, sense=pyo.maximize)

solver = pyo.SolverFactory(modules.find('bonmin'), solver_io='nl')
result = solver.solve(model)

optimized_prices = [np.round(pyo.value(model.prices[i])).astype(int) for i in range(n_product)]
deal = [np.round(pyo.value(model.deal[i])).astype(int) for i in range(n_product)]
# deal_days = [np.round(pyo.value(model.deal_days[i])).astype(int) for i in model.A]

df_result = df_params.copy()
df_result['optimized_price'] = optimized_prices
df_result['expected_sales'] = (
    df_result['slope'] * df_result['optimized_price'] + df_result['intercept']).round(0).astype(int)
df_result['expected_sales_deal'] = (
    df_result['deal_slope'] * df_result['optimized_price'] + df_result['deal_intercept']).round(0).astype(int)
df_result['fba_commission_local'] = df_result['optimized_price']*0.15
df_result['labor_cost_6_percent'] = df_result['optimized_price']*0.06
df_result['storage_cost_2_percent'] = df_result['optimized_price']*0.02
# 是否做deal
df_result['is_deal'] = deal
df_result['product_income'] = df_result['optimized_price'] * (1 - df_result['damage_rate']) - df_result['latest_amazon_delivery_fee_usd'] - \
    df_result['fba_commission_local'] - df_result['vat_local'] - df_result['promotion_discount'] - \
    df_result['ppc_cost'] - df_result['labor_cost_6_percent'] - \
    df_result['storage_cost_2_percent']
df_result['product_profit_rmb'] = df_result['product_income'] * df_result['exchange_rate'] - df_result['purchase_cost'] - \
    df_result['shipping_and_tax']
df_result['product_income_deal'] = df_result['optimized_price'] * 0.85 * (1 - df_result['damage_rate']) - df_result['latest_amazon_delivery_fee_usd'] - \
    df_result['fba_commission_local'] - df_result['vat_local'] - df_result['promotion_discount'] - \
    df_result['ppc_cost'] - df_result['labor_cost_6_percent'] - \
    df_result['storage_cost_2_percent']
df_result['product_profit_rmb_deal'] = df_result['product_income_deal'] * df_result['exchange_rate'] - df_result['purchase_cost'] - \
    df_result['shipping_and_tax']
# df_result['month_profit_rmb'] = (30 - deal_days[0] - deal_days[1]*7) * df_result['product_profit_rmb'] * df_result['expected_sales'] \
#     + deal_days[0] * df_result['product_profit_rmb_deal'] * df_result['expected_sales_deal'] \
#     - deal_days[0]*150 - deal_days[1]*300

df_result = df_result.drop(
    columns=['slope', 'intercept', 'deal_slope', 'deal_intercept'])

In [50]:
model.display()

Model unknown

  Variables:
    prices : Size=18, Index={0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17}
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :    34 :  35.0 :    35 : False : False : PositiveIntegers
          1 :    34 :  40.0 :    40 : False : False : PositiveIntegers
          2 :    34 :  45.0 :    45 : False : False : PositiveIntegers
          3 :    34 :  39.0 :    41 : False : False : PositiveIntegers
          4 :    34 :  40.0 :    42 : False : False : PositiveIntegers
          5 :    34 :  42.0 :    43 : False : False : PositiveIntegers
          6 :    34 :  44.0 :    44 : False : False : PositiveIntegers
          7 :    34 :  39.0 :    45 : False : False : PositiveIntegers
          8 :    34 :  44.0 :    46 : False : False : PositiveIntegers
          9 :    34 :  47.0 :    47 : False : False : PositiveIntegers
         10 :    34 :  47.0 :    48 : False : False : PositiveIntegers
         11 :    34 :  49.0 :    49 : F

In [19]:
from src.opt import preprocess_file, optimize, result_process

df_params = preprocess_file(df_params)
optimized_prices, deal = optimize(df_params)
df_result = result_process(df_params, optimized_prices, deal)

In [20]:
df_result

,model,purchase_cost,shipping_and_tax,latest_amazon_delivery_fee_usd,vat_local,damage_rate,promotion_discount,ppc_cost,exchange_rate,price_lower_bound,...,expected_sales_deal,fba_commission_local,labor_cost_6_percent,storage_cost_2_percent,is_deal,product_income,product_profit_rmb,product_income_deal,product_profit_rmb_deal,month_profit_rmb
0,0BLK,65,10.00,7.09,0,0.1049,0,2,7.1,34,...,254,5.10,2.04,0.68,10,13.5234,21.01614,8.958390,-11.395431,24436.60086
1,0BLU,65,10.00,7.09,0,0.1213,0,0,7.1,34,...,40,6.00,2.40,0.80,0,18.8580,58.89180,13.585800,21.459180,35335.08000
2,0PUR,67,10.00,7.09,0,0.1218,0,0,7.1,34,...,9,6.60,2.64,0.88,0,21.4308,75.15868,15.634680,34.006228,9019.04160
3,0BBU,65,10.00,7.09,0,0.0940,0,0,7.1,34,...,6,5.55,2.22,0.74,0,17.9220,52.24620,12.893700,16.545270,4702.15800
4,0PNK,63,10.00,7.17,0,0.0817,0,0,7.1,34,...,6,5.70,2.28,0.76,0,18.9854,61.79634,13.751090,24.632739,5561.67060
5,0WHT,63,10.00,7.17,0,0.1239,0,0,7.1,34,...,3,6.00,2.40,0.80,0,18.6740,59.58540,13.417400,22.263540,1787.56200
6,0PBU,69,10.00,7.17,0,0.1161,0,0,7.1,34,...,9,6.45,2.58,0.86,0,20.9477,69.72867,15.246545,29.250469,8367.44040
7,0AGN,69,10.00,7.17,0,0.0815,0,0,7.1,34,...,18,5.40,2.16,0.72,0,17.6160,46.07360,12.656100,10.858310,12439.87200
8,0LPR,69,10.00,7.17,0,0.0765,0,0,7.1,34,...,5,6.30,2.52,0.84,0,21.9570,76.89470,16.138950,35.586545,6920.52300
9,2BLK,65,10.00,6.77,0,0.1513,0,0,7.1,34,...,21,6.90,2.76,0.92,0,21.6902,79.00042,15.834170,37.422607,26070.13860
